In [ ]:
import sympy
import numpy as np
import random
from tqdm import tqdm
from fractions import Fraction

bits = 40
#generate prime number with given bit length
flag = True
while flag:
    b=random.getrandbits(bits)
    p = sympy.nextprime(b)
    bin_p = bin(p)[2:]
    if len(bin_p)==bits:
        flag=False
max_len = len(bin_p)
#select a
below=0
while(below==0):
    a = random.getrandbits(max_len)
    if(a<=p-1):
        below=1 
print(p)
print(a)
print(max_len)


X = []
Y = []
a = Fraction(a)
for i in range(500000):
    x = Fraction(random.randint(1, p-1))
    result = bin(int((a*x)%p))[2:]
    result = "0"*(max_len-len(result))+result
    x_result = bin(int(x))[2:]
    x_result = "0"*(max_len-len(x_result))+x_result
    X.append([int(el) for el in result])
    Y.append([int(el) for el in x_result])
    
X = np.array(X, dtype='float32')
Y = np.array(Y, dtype='float32')
print(X.shape)
print(Y.shape)

In [ ]:
import torch
from torch import nn
from torch.nn import functional as F
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader

class BaseModel1(nn.Module):
    def __init__(self, ):
        super(BaseModel1, self).__init__()
        self.linear1 = nn.Linear(max_len, 1000) 
        self.linear2 = nn.Linear(1000, 1000)
        self.linear3 = nn.Linear(1000, max_len)

    def forward(self, inputs):
        h1 = torch.sigmoid(self.linear1(inputs))
        h2 = torch.sigmoid(self.linear2(h1))
        logits = self.linear3(h2)
        return logits 

batch_size = 1000

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

X_train = torch.from_numpy(X_train)
y_train = torch.from_numpy(y_train)
X_test = torch.from_numpy(X_test) 
y_test = torch.from_numpy(y_test)


train_dataset = TensorDataset(X_train,y_train)
train_loader = DataLoader(train_dataset, batch_size = batch_size)

test_dataset = TensorDataset(X_test,y_test)
test_loader = DataLoader(test_dataset, batch_size = batch_size)


In [ ]:
EPOCHS = 6000
model = BaseModel1()
model.cuda()
criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
data = []
train_chunks = int(np.ceil(len(X_train)/batch_size))
test_chunks = int(np.ceil(len(X_test)/batch_size))

#train model
for i in range(EPOCHS):
    train_correct = 0
    total_train_loss = 0
    total_test_loss = 0
    test_correct = 0
    train_bit_correct = np.zeros(max_len)
    test_bit_correct = np.zeros(max_len)
    for j in range(train_chunks):
        x = X_train[j*batch_size:(j+1)*batch_size]
        y = y_train[j*batch_size:(j+1)*batch_size]

        model.zero_grad()
        optimizer.zero_grad()
        x = x.cuda()
        y = y.cuda()    
        logits = model(x)
        #sum of binary cross-entropy losses
        loss = criterion(logits[:,0], y[:,0])
        for z in range(1, max_len):
            loss += criterion(logits[:,z], y[:,z])
        loss.backward()
        optimizer.step()
        total_train_loss += loss.item()
        train_correct += ((y==(logits>0).to(float)).sum(axis=1)/max_len).sum().item()
        train_bit_correct += (y==(logits>0).to(float)).sum(axis=0).cpu().numpy()
        

    with torch.no_grad():
        for j in range(test_chunks):
            x = X_test[j*batch_size:(j+1)*batch_size]
            y = y_test[j*batch_size:(j+1)*batch_size]
            x = x.cuda()
            y = y.cuda()    

            logits = model(x)
            loss = criterion(logits[:,0], y[:,0])
            for z in range(1, max_len):
                loss += criterion(logits[:,z], y[:,z])
            total_test_loss += loss.item()
            test_correct += ((y==(logits>0).to(float)).sum(axis=1)/max_len).sum().item()
            test_bit_correct += (y==(logits>0).to(float)).sum(axis=0).cpu().numpy()


    train_acc = train_correct/len(X_train)
    train_loss = total_train_loss/len(X_train)
    test_loss = total_test_loss/len(X_test)
    test_acc = test_correct/len(X_test)
    data.append([train_acc, train_loss, test_loss, test_acc]+list(train_bit_correct/len(X_train))+list(test_bit_correct/len(X_test)))
    if i % 20==0:
        print(i,"train_acc:", round(train_acc, 6), 'train_loss:',
              round(train_loss, 7), 'val_loss:', round(test_loss, 6), 
              "val_acc:", round(test_acc, 6))
    if (test_acc>=1) and (train_acc>=1):
        break


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

bit_train_cols = ['bit_train_'+str(i) for i in list(range(max_len,0,-1))]
bit_test_cols = ['bit '+str(i) if i>=36 else "_bit "+str(i) for i in list(range(max_len,0,-1))]
df = pd.DataFrame(data, columns = ['train_acc', 'train_loss', 'test_loss', 'test_acc']\
                 + bit_train_cols + bit_test_cols)
df = df.reset_index().rename({'index':'step'}, axis=1)

ax = df.plot(x='step', y=bit_test_cols)
plt.xlabel('Step')
plt.ylabel('Accuracy')
plt.rcParams.update({'font.size': 14})
ax.legend(bbox_to_anchor=(1.22, 1.02), prop = { "size": 10 })
plt.savefig('40_bits.png', dpi=300, bbox_inches = 'tight')